In [1]:
reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:

%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from lxml import etree
import requests
from IPython.display import display, HTML

In [3]:
headers = {
    'x-devtools-emulate-network-conditions-client-id': "c60c3ba6-26e8-45ce-9f9f-0b5521437cf6",
    'upgrade-insecure-requests': "1",
    'user-agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36",
    'accept': "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    'referer': "http://isa.epfl.ch/imoniteur_ISAP/%21gedpublicreports.htm?ww_i_reportmodel=133685247",
    'accept-encoding': "gzip, deflate, sdch",
    'accept-language': "es-ES,es;q=0.8",
    'cookie': "LANGUE_LOGIN=fr",
    'cache-control': "no-cache"
}

data = requests.get("http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_i_reportModel=133685247", headers=headers)

soup = BeautifulSoup(data.content, 'html.parser')
queries = soup.find_all('select')
queries_map = {}
for s in queries:
    queries_map[s.get('name')] = {}
    map = queries_map[s.get('name')]
    options = s.find_all('option')
    for o in options:
        if not o.string:
            value = ""
        else:
            value = o.string
        map[value] = o.get('value')

print(queries_map)

{'ww_x_PERIODE_PEDAGO': {'': 'null', 'Bachelor semestre 5b': '2226768', 'Bachelor semestre 1': '249108', 'Bachelor semestre 6b': '2226785', 'Stage printemps master': '2227132', 'Projet Master automne': '249127', 'Master semestre 1': '2230106', 'Bachelor semestre 6': '942175', 'Bachelor semestre 2': '249114', 'Mineur semestre 2': '2335676', 'Stage automne 4ème année': '2226616', 'Mise à niveau': '2063602308', 'Master semestre 3': '2230128', 'Semestre automne': '953159', 'Stage printemps 4ème année': '2226626', 'Stage automne 3ème année': '953137', 'Bachelor semestre 4': '942163', 'Bachelor semestre 5': '942120', 'Mineur semestre 1': '2335667', 'Bachelor semestre 3': '942155', 'Projet Master printemps': '3781783', 'Master semestre 4': '2230140', 'Master semestre 2': '942192', 'Stage printemps 3ème année': '983606', 'Semestre printemps': '2754553'}, 'ww_x_UNITE_ACAD': {'': 'null', 'Mathématiques': '944590', 'Section FCUE': '1574548993', 'Systèmes de communication': '946228', 'Génie électr

In [4]:
def getData(unite_acad, periode_acad, periode_pedago, semester_type):
    unite_acad_id = queries_map['ww_x_UNITE_ACAD'][unite_acad]
    periode_acad_id = queries_map['ww_x_PERIODE_ACAD'][periode_acad]
    periode_pedago_id = queries_map['ww_x_PERIODE_PEDAGO'][periode_pedago]
    semester_type_id = queries_map['ww_x_HIVERETE'][semester_type]
    url = "http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter"

    querystring = {"ww_b_list":"1",
                   "ww_i_reportmodel":"133685247",
                   "ww_c_langue":"",
                   "ww_i_reportModelXsl":"133685270",
                   "zz_x_UNITE_ACAD": unite_acad,
                   "ww_x_UNITE_ACAD": unite_acad_id,
                   "zz_x_PERIODE_ACAD": periode_acad,
                   "ww_x_PERIODE_ACAD":periode_acad_id,
                   "zz_x_PERIODE_PEDAGO":periode_pedago,
                   "ww_x_PERIODE_PEDAGO":periode_pedago_id,
                   "zz_x_HIVERETE":semester_type,
                   "ww_x_HIVERETE":semester_type_id,
                   "dummy":"ok"}

    headers = {
        'upgrade-insecure-requests': "1",
        'user-agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36",
        'accept': "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        'referer': "http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_i_reportModel=133685247",
        'accept-encoding': "gzip, deflate, sdch",
        'accept-language': "es-ES,es;q=0.8",
        'cookie': "LANGUE_LOGIN=fr",
        'cache-control': "no-cache",
        'postman-token': "65d96955-59bb-daa2-5136-807135f5b89a"
        }

    text = BeautifulSoup(requests.request("GET", url, headers=headers, params=querystring).text, 'html.parser')
    gps = list(text.find_all('table'))[2].find_all('tr')[1].td.a.get('onclick').split('ww_x_GPS=')[1].split("')")[0]
    payload = {'ww_x_GPS': int(gps),
               'ww_i_reportModel':133685247, 
               'ww_i_reportModelXsl': 133685271,
               'ww_x_UNITE_ACAD': unite_acad_id,
               'ww_x_PERIODE_ACAD': periode_acad_id,
               'ww_x_PERIODE_PEDAGO': periode_pedago_id,
               'ww_x_HIVERETE': semester_type_id
              }
    r = requests.get("http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.XLS", params=payload)
    return r.text

In [5]:
df_final = None
df_year = None

def addData(unite_acad, periode_acad, periode_pedago, semester_type, list_semesters):
    unite_acad_id = queries_map['ww_x_UNITE_ACAD'][unite_acad]
    periode_acad_id = queries_map['ww_x_PERIODE_ACAD'][periode_acad]
    periode_pedago_id = queries_map['ww_x_PERIODE_PEDAGO'][periode_pedago]
    semester_type_id = queries_map['ww_x_HIVERETE'][semester_type]

    
    soup = BeautifulSoup(getData(unite_acad,periode_acad,periode_pedago,semester_type), 'html.parser')
    rows = soup.find_all('tr')
    header_cols = rows[2].find_all('th')
    col_titles, data = [], {}
    
    for c in header_cols:
        data[c.string] = []
        col_titles.append(c.string)
    [col_titles.append(semestre[9:]) for semestre in list_semesters]
    for semestre in list_semesters:
        data[semestre[9:]] = [ float('nan') ] * ( len(rows) - 3)
    col_titles.append('enter')
    data['enter'] = None
    col_titles.append('exit')
    data['exit'] = None
    global df_final, df_year


    df = pd.DataFrame(columns=col_titles)

    df.set_index(['No Sciper'], inplace=True, drop=False)
    if df_final is None:
        df_final = df.copy()
    if df_year is None:
        df_year = df.copy() 

    for h in range(3, len(rows)):
        values = rows[h].find_all('td')
        for v in range(len(values)-2):
            data[header_cols[v].string].append(values[v].string)
        data[periode_pedago[9:]][h-3] =  True 
    df = pd.DataFrame.from_dict(data)
    df.set_index(['No Sciper'], inplace=True, drop=False)
    return df



In [6]:
def combineDataYear(new_data, old_data, semestre, year):
    print(semestre, year)
    for user in new_data['No Sciper'].tolist():
        if (user in old_data['No Sciper'].tolist()):
            #print(old_data[semestre[9:]].loc[old_data['No Sciper'] == user], "*******")
            old_data[semestre[9:]].loc[old_data['No Sciper'] == user] = True
            old_data['exit'].loc[old_data['No Sciper'] == user] = year
        else:
            new_data['enter'].loc[new_data['No Sciper'] == user] = year
            new_data['exit'].loc[new_data['No Sciper'] == user] = year
            old_data = pd.concat([old_data, new_data.loc[new_data['No Sciper'] == user]], ignore_index=True)
            #print(new_data['semestre 1'].loc[new_data['No Sciper'] == user])
    old_data.set_index(['No Sciper'], inplace=True, drop=False)


    return old_data

def combineDataTotal(new_data, old_data):
    old_data = old_data.combine_first(new_data)
    for user in new_data['No Sciper'].tolist():
        #if user in old_data['No Sciper'].tolist():
        #    old_data.loc[old_data['No Sciper'] == user] = old_data.loc[old_data['No Sciper'] == user].combine_first(new_data.loc[new_data['No Sciper'] == user])
            
            
            #old_data.update(new_data, overwrite=True)
        #else:
        if user not in old_data['No Sciper'].tolist():
            old_data = pd.concat([old_data, new_data.loc[new_data['No Sciper'] == user]])
        else:
            old_data['exit'].loc[old_data['No Sciper'] == user] = new_data['exit'].loc[new_data['No Sciper'] == user]
    old_data.set_index(['No Sciper'], inplace=True, drop=False)

    return old_data

In [7]:
unite_acad = "Informatique"
periode_acad = list(queries_map['ww_x_PERIODE_ACAD'].keys())
periode_acad.remove('')
periode_pedago = []
for p in list(queries_map['ww_x_PERIODE_PEDAGO'].keys()):
    if 'Bachelor' in p and "b" not in p:
        periode_pedago.append(p)
periode_pedago = sorted(periode_pedago)
periode_acad = sorted(periode_acad)
df_final = None
df_year = None


for year in range(len(periode_acad)):
    df_year = None
    for s in range(len(periode_pedago)):
        a = addData("Informatique", periode_acad[year], periode_pedago[s], "", periode_pedago)
        df_year = combineDataYear(a, df_year, periode_pedago[s], periode_acad[year])
        #display(df_year)
    df_final = combineDataTotal(df_year, df_final)
    
display(df_final)


#df_final = df_final.combine_first(df_year)


Bachelor semestre 1 2007-2008


/Users/nachoaguadoc/anaconda/lib/python3.5/site-packages/pandas/core/indexing.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Bachelor semestre 2 2007-2008
Bachelor semestre 3 2007-2008
Bachelor semestre 4 2007-2008
Bachelor semestre 5 2007-2008
Bachelor semestre 6 2007-2008
Bachelor semestre 1 2008-2009
Bachelor semestre 2 2008-2009
Bachelor semestre 3 2008-2009
Bachelor semestre 4 2008-2009
Bachelor semestre 5 2008-2009
Bachelor semestre 6 2008-2009
Bachelor semestre 1 2009-2010
Bachelor semestre 2 2009-2010
Bachelor semestre 3 2009-2010
Bachelor semestre 4 2009-2010
Bachelor semestre 5 2009-2010
Bachelor semestre 6 2009-2010
Bachelor semestre 1 2010-2011
Bachelor semestre 2 2010-2011
Bachelor semestre 3 2010-2011
Bachelor semestre 4 2010-2011
Bachelor semestre 5 2010-2011
Bachelor semestre 6 2010-2011
Bachelor semestre 1 2011-2012
Bachelor semestre 2 2011-2012
Bachelor semestre 3 2011-2012
Bachelor semestre 4 2011-2012
Bachelor semestre 5 2011-2012
Bachelor semestre 6 2011-2012
Bachelor semestre 1 2012-2013
Bachelor semestre 2 2012-2013
Bachelor semestre 3 2012-2013
Bachelor semestre 4 2012-2013
Bachelor s

,Civilité,Ecole Echange,Filičre opt.,Mineur,No Sciper,Nom Prénom,Orientation Bachelor,Orientation Master,Spécialisation,Statut,Type Echange,enter,exit,semestre 1,semestre 2,semestre 3,semestre 4,semestre 5,semestre 6
No Sciper,,,,,,,,,,,,,,,,,,,
147008,Monsieur,Carnegie Mellon University Pittsburgh,NaN,NaN,147008,Good Xavier,NaN,NaN,NaN,Présent,Bilatéral,2008-2009,2010-2011,True,True,True,True,True,True
153762,Monsieur,NaN,NaN,NaN,153762,Conus Johann,NaN,NaN,NaN,Présent,NaN,2007-2008,2007-2008,NaN,NaN,NaN,NaN,True,True
154157,Madame,NaN,NaN,NaN,154157,Andriambololona Riana Miarantsoa,NaN,NaN,NaN,Présent,NaN,2007-2008,2007-2008,NaN,NaN,NaN,NaN,True,NaN
159516,Monsieur,NaN,NaN,NaN,159516,Raja Yanick,NaN,NaN,NaN,Présent,NaN,2007-2008,2007-2008,NaN,NaN,NaN,NaN,True,True
159998,Madame,NaN,NaN,NaN,159998,Jesse Julia,NaN,NaN,NaN,Présent,NaN,2007-2008,2007-2008,NaN,NaN,NaN,NaN,True,True
160213,Monsieur,NaN,NaN,NaN,160213,Bécholey Alexandre,NaN,NaN,NaN,Présent,NaN,2007-2008,2007-2008,NaN,NaN,NaN,NaN,True,NaN
161091,Madame,NaN,NaN,NaN,161091,Grivet Ekaterina,NaN,NaN,NaN,Congé,NaN,2007-2008,2007-2008,NaN,NaN,NaN,NaN,True,True
161127,Monsieur,NaN,NaN,NaN,161127,Sahy François,NaN,NaN,NaN,Congé,NaN,2007-2008,2007-2008,NaN,NaN,NaN,NaN,True,True
161212,Monsieur,NaN,NaN,NaN,161212,Tamesna Rachid,NaN,NaN,NaN,Présent,NaN,2009-2010,2009-2010,True,True,NaN,NaN,NaN,NaN


hola

In [11]:
df_final.drop(['Ecole Echange', 'Filičre opt.', 'Mineur', 'Orientation Bachelor','Orientation Master', 'Spécialisation', 'Type Echange' ],1)



,Civilité,No Sciper,Nom Prénom,Statut,enter,exit,semestre 1,semestre 2,semestre 3,semestre 4,semestre 5,semestre 6
No Sciper,,,,,,,,,,,,
147008,Monsieur,147008,Good Xavier,Présent,2008-2009,2010-2011,True,True,True,True,True,True
153762,Monsieur,153762,Conus Johann,Présent,2007-2008,2007-2008,NaN,NaN,NaN,NaN,True,True
154157,Madame,154157,Andriambololona Riana Miarantsoa,Présent,2007-2008,2007-2008,NaN,NaN,NaN,NaN,True,NaN
159516,Monsieur,159516,Raja Yanick,Présent,2007-2008,2007-2008,NaN,NaN,NaN,NaN,True,True
159998,Madame,159998,Jesse Julia,Présent,2007-2008,2007-2008,NaN,NaN,NaN,NaN,True,True
160213,Monsieur,160213,Bécholey Alexandre,Présent,2007-2008,2007-2008,NaN,NaN,NaN,NaN,True,NaN
161091,Madame,161091,Grivet Ekaterina,Congé,2007-2008,2007-2008,NaN,NaN,NaN,NaN,True,True
161127,Monsieur,161127,Sahy François,Congé,2007-2008,2007-2008,NaN,NaN,NaN,NaN,True,True
161212,Monsieur,161212,Tamesna Rachid,Présent,2009-2010,2009-2010,True,True,NaN,NaN,NaN,NaN


Now....

In [26]:
df_final_sm1 = df_final[df_final['semestre 1'] == True]
df_final = df_final_sm1[df_final_sm1['semestre 6'] == True]
num_total = len(df_final)

In [27]:
num_women = len(df_final[df_final['Civilité'] == 'Madame'])

In [ ]:
from pylab import *

# make a square figure and axes
figure(1, figsize=(6,6))
ax = axes([0.1, 0.1, 0.8, 0.8])

# The slices will be ordered and plotted counter-clockwise.
labels = 'Men', 'Women'
fracs = [num_total-num_women, num_women]
explode=(0, 0, 0, 0)

pie(fracs, explode=explode, labels=labels,
                autopct='%1.1f%%', shadow=True, startangle=90)

title('Raining Hogs and Dogs', bbox={'facecolor':'0.8', 'pad':5})

show()